<a href="https://colab.research.google.com/github/kampelmuehler/MLKurs/blob/main/image_recognition/SemanticSegmentationDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#@title Setup
import torch
from torchvision import models
from PIL import Image
import urllib
import ast
import matplotlib.pyplot as plt


weights = models.segmentation.DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT
transform = models.segmentation.DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1.transforms()
model = models.segmentation.deeplabv3_mobilenet_v3_large(weights=weights)
model.eval()

def read_image(url):
  img = Image.open(urllib.request.urlopen(url)).convert("RGB")
  thumbnail = img.copy()
  thumbnail.thumbnail((400, 400), Image.ANTIALIAS)
  display(thumbnail)
  return transform(img).unsqueeze(0), thumbnail.size

palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# COCO Semantische Segmentierung

Hier könnt ihr ein auf Resnet50 basierendes DeepLabV3 Modell, welches auf COCO, ausprobieren. Einfach eine url zu einem Bild einfügen.

In [ ]:
url = ""
img, thumbnail_size = read_image(url)
result = model(img)['out'][0].argmax(0)


# plot the semantic segmentation predictions of 21 classes in each color
result_img = Image.fromarray(result.byte().cpu().numpy()).resize(thumbnail_size)
result_img.putpalette(colors)
display(result_img)